In [1]:
import tensorflow as tf
import utils
import numpy as np

__author__ = "Olivares Castillo José Luis"
tf.__version__


'1.4.0'

In [2]:
es,na = utils.load_node2vec()

In [3]:
es.shape[1]

129

In [4]:
index_es, index_na = utils.get_seed_index(es, na)
es_vectores = utils.get_vectors(es, index_es)
na_vectores = utils.get_vectors(na, index_na)

In [5]:
index_es

[734,
 4199,
 3508,
 3047,
 2645,
 3174,
 1157,
 751,
 1574,
 3262,
 3421,
 1207,
 1475,
 2972,
 4312,
 1034,
 2239,
 2487,
 103,
 2939,
 2365,
 2780,
 1756,
 1745,
 602,
 3149,
 793,
 619,
 1948,
 3113,
 1352,
 3609,
 2321,
 3879,
 1359,
 1415,
 2794,
 3420,
 2923,
 3745,
 948,
 2650,
 4071,
 2347,
 1143,
 3212,
 3637,
 20,
 1973,
 3658,
 1121,
 2694,
 3683,
 1205,
 4409,
 2187,
 576,
 1146,
 2956,
 37,
 3676,
 4262,
 3442,
 3756,
 1772,
 100,
 2248,
 2336,
 1927,
 823,
 2332,
 1102,
 3962,
 443,
 3863,
 1389,
 1269,
 75,
 1641,
 1155,
 4144,
 2869,
 434,
 2242,
 3154,
 3612,
 2889,
 3519,
 2753,
 886,
 4121,
 241,
 1693,
 136,
 1206,
 4008,
 1534,
 3808,
 2886,
 1639,
 3991,
 3375,
 4098,
 1054,
 2655,
 3846,
 2801,
 4304,
 7,
 2261,
 4349,
 701,
 157,
 728,
 2767,
 2414,
 2948,
 2560,
 4329,
 761,
 1353,
 743,
 2458,
 3554,
 2392,
 4160,
 3704,
 1064,
 3799,
 3579,
 3486,
 171,
 2731,
 2734,
 3680,
 2267,
 2085,
 2368,
 3951,
 2484,
 855,
 2666,
 2453,
 1232,
 1856,
 2879,
 3985,
 2

In [6]:
lexes=[es.iloc[_][0] for _ in index_es]
lexna=[na.iloc[_][0] for _ in index_na]

In [7]:
new_lexicon =list(zip(lexes,lexna))

In [8]:
#for _ in new_lexicon:
#    print(_[0],_[1])

In [9]:
es[es[0] == "y"][0]

Series([], Name: 0, dtype: object)

In [10]:
es[es[0] == "se"][0]

Series([], Name: 0, dtype: object)

In [11]:
na[na[0] == "no"][0]

Series([], Name: 0, dtype: object)

In [12]:
es_dummy = es.drop(es.columns[0],axis=1)
na_dummy = na.drop(na.columns[0],axis=1)
es_vectores1 = np.array(es_dummy)
na_vectores1 = np.array(na_dummy)
print(na_vectores1.shape)

(1904, 128)


In [13]:
#es_vectores = [es.iloc[_].loc[1::] for _ in range(es.shape[0])]

In [14]:
print(es.iloc[3421][0])

sal


In [15]:
# Palabra y vector
#indice = 1756
#print(es.iloc[3508][0])
test_vectors = np.array([np.array(es.iloc[indice][1::]).astype(np.float64) for indice in index_es])
#test_vectors = na_vectores1
#test_vectors = np.array(es.iloc[3421][1::]).astype(np.float64)
#test_vectors.resize(1,128)
print(test_vectors.shape)

(540, 128)


In [16]:
sess = tf.Session()

# Modelos

## Test 1

* `LEARNING_RATE = 0.4666`
* `NODES_H1 = 300`
* `XAVIER_INIT = False`
* `EPOCHS = 370000`
* `fc1 = activation_function(fc1, "relu", "fc1")`
* `nah_predicted = activation_function(output, "sigmoid", "output")`
* Loss Function = MSE
* `optimiser = tf.train.AdagradOptimizer(learning_rate=LEARNING_RATE)` W/ gradient clipping.

## Test 2

* `LEARNING_RATE = 1.0` [Ver liga](https://stackoverflow.com/questions/43892849/keras-batchnorm-training-accuracy-increases-while-testing-accuracy-decreases)
* `NODES_H1 = 90`
* `XAVIER_INIT = True`
* `EPOCHS = 370000`
* `fc1 = activation_function(fc1, "sigmoid", "fc1")`
* `nah_predicted = fully_connected_layer(fc1, NODES_H1, NODES_OUPUT, "output",xavier_init=XAVIER_INIT)`
**NOTA:** No se activa la capa de salida.
* Loss Function = MSE
* `optimiser = tf.train.MomentumOptimizer(learning_rate=LEARNING_RATE,momentum=0.5)` W/ gradient clipping.

In [17]:
saver = tf.train.import_meta_graph('./models/model2/Momentum_H_90_LR_1.0.ckpt.meta')
saver.restore(sess,tf.train.latest_checkpoint('./models/model2/'))

INFO:tensorflow:Restoring parameters from ./models/model2/Momentum_H_90_LR_1.0.ckpt


In [18]:
graph = tf.get_default_graph()

In [19]:
#print(sess.run("output/W:0"))

In [20]:
X = graph.get_tensor_by_name("input/input_es:0")
#y = graph.get_tensor_by_name("input/target_na:0")

In [21]:
print(X.shape)

(?, 128)


In [22]:
feed_dict = {X:test_vectors}

In [23]:
#Salida modelo 2
output_NN = graph.get_tensor_by_name("output/xw_plus_b:0")

In [24]:
pred = sess.run(output_NN,feed_dict)
print (type(pred[0]),pred.shape)

<class 'numpy.ndarray'> (540, 128)


In [25]:
#print(pred[0].shape)

In [26]:
top_10=[utils.get_top10_closest(pred[_],na_vectores1) for _ in range(pred.shape[0])]
#top_10=utils.get_top10_closest(pred[0],na_vectores1)
#top_10

In [27]:
closest = [utils.get_closest_words_to(top_10[_],na) for _ in range(pred.shape[0])]
#closest = utils.get_closest_words_to(top_10,na)

In [28]:
closest[10]

['gali',
 'año',
 'xihuitl',
 'yb',
 'diego',
 'dral',
 'rosario',
 'tepahpaquiltia',
 'xiuh',
 'federal']

In [29]:
na_words = list(na[0])
len(na_words)

1904

In [30]:
(lexna[10]),closest[10]

('xihuitl',
 ['gali',
  'año',
  'xihuitl',
  'yb',
  'diego',
  'dral',
  'rosario',
  'tepahpaquiltia',
  'xiuh',
  'federal'])

In [31]:
type(closest[0][0])

str

In [32]:
p_at_k = list()

for k in [1,5,10]:
    p_at_k.append([utils.apk(lexna[_],closest[_],k) for _ in range(len(lexna))])

In [46]:
p_at_k[1]

[0.2,
 0.25,
 0.5,
 0.2,
 0.2,
 0.2,
 0.3333333333333333,
 0.25,
 0.2,
 0.2,
 0.06666666666666667,
 0.25,
 0.3333333333333333,
 0.2,
 0.2,
 0.2,
 0.4,
 0.3333333333333333,
 0.25,
 0.3,
 0.2,
 0.3,
 0.0,
 0.2,
 0.2,
 0.2,
 0.25,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.25,
 0.2,
 0.0,
 0.2,
 0.3333333333333333,
 0.2,
 0.2,
 0.25,
 0.2,
 0.25,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.0,
 0.2,
 0.4,
 0.25,
 0.3333333333333333,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.05,
 0.25,
 0.2,
 0.25,
 0.2,
 0.3333333333333333,
 0.25,
 0.2,
 0.2,
 0.3333333333333333,
 0.3333333333333333,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.25,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.3333333333333333,
 0.25,
 0.25,
 0.2,
 0.2,
 0.4,
 0.2,
 0.4,
 0.2,
 0.2,
 0.25,
 0.2,
 0.41666666666666663,
 0.2,
 0.2,
 0.2,
 0.25,
 0.2,
 0.2,
 0.3333333333333333,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.0,
 0.25,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.2,
 0.0,
 0.2,
 0.2,
 0.2,
 0.2,
 

In [39]:
#acc1=[(_,lexna[_] == closest[_][0]) for _ in range(pred.shape[0])]
acc=[(lexna[_] == closest[_][0]) for _ in range(pred.shape[0])]

In [40]:
acc

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,

In [41]:
precision_list = np.array(acc).astype(np.int)
precision_list

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1,

In [42]:
from collections import Counter

In [43]:
precision = Counter(precision_list)

In [44]:
precision

Counter({0: 24, 1: 516})

In [45]:
print("Precisión con lexicon de entrenamiento:",(precision[1]/len(precision_list))*100)

Precisión con lexicon de entrenamiento: 95.55555555555556
